# Calculate Phi using pO2_bar, T_bar

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE-FOSI_hindcast_O2_T_S_mean_dimension_reduced_1948-2017_monthly.nc'
ds1 = xr.open_dataset(filename)
T_mean = ds1['T_mean']
##
filename = f'{path}/DPLE-FOSI_hindcast_pO2_mean_dimension_reduced_1948-2017_monthly.nc'
ds1 = xr.open_dataset(filename)
pO2_mean = ds1['pO2_mean']

## Calculate Φ using 9 different combinations of Ac, Eo from Traits_db

In [3]:
import traits_db
from scipy import stats as scistats

In [4]:
df = traits_db.df_mi
df.head()

,Species,Phylum,Ao,Ac,Eo
0,Acanthephyra acutifrons,Crustacea,26.809651,13.364732,-0.0278
1,Acanthephyra curtirostris,Crustacea,25.575448,14.364194,0.0380
2,Acanthephyra purpurea,Crustacea,23.310023,7.460879,0.1320
3,Acanthephyra smithi,Crustacea,28.985507,17.657939,-0.0301
5,Acipenser brevirostrum,Chordata,22.123894,4.901716,0.3626


In [5]:
Ac_dist = np.percentile(df.Ac, [50])
#Ao_dist = np.percentile(df.Ao, [25, 50, 75])
Eo_dist = np.percentile(df.Eo, [25, 50, 75])
print(Ac_dist)
#print(Ao_dist)
print(Eo_dist)

[7.34832416]
[0.1985 0.3389 0.5625]


### calculate Φ based on the 9 combinations - O2 const

In [6]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product
dss = xr.Dataset()

for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    dss[f'Phi_{3*i+j}'] = mi.Phi(pO2_mean, T_mean, Ac, Eo, dEodT=0.022)
    # = xr_reshape(ppphi,'time',['year','mon'],[np.arange(70),np.arange(12)]).mean(dim='mon')
dss

7.3483241595629805 0.1985
7.3483241595629805 0.3389
7.3483241595629805 0.5625
CPU times: user 216 ms, sys: 166 ms, total: 382 ms
Wall time: 737 ms


<xarray.Dataset>
Dimensions:  (z_t: 60, nlat: 384, nlon: 320)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG    (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 318.9 319.4 319.8
    ULAT     (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG    (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 319.2 319.6 320.0
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_0    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_1    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_2    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan

In [7]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dss.load()

<xarray.Dataset>
Dimensions:  (z_t: 60, nlat: 384, nlon: 320)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG    (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 318.9 319.4 319.8
    ULAT     (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG    (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 319.2 319.6 320.0
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_0    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_1    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_2    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan

In [8]:
%%time
dss.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_mean_dimesion_reduced_1948-2017_monthly.nc', mode='w')

CPU times: user 57.4 ms, sys: 25.5 ms, total: 82.9 ms
Wall time: 263 ms
